In [1]:
from pyspark import SparkContext #Library
sc = SparkContext("local[*]", "temp") #Start a spart context


In [2]:
posts_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allPosts/*.gz") #create an RDD from file
print posts_lines
posts_lines.count()
#posts_lines.take(8)

MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:-2


212990

In [3]:
votes_lines = sc.textFile("file:///home/vagrant/miniprojects/spark/part/allVotes/*.gz")
print votes_lines
votes_lines.count()

MapPartitionsRDD[4] at textFile at NativeMethodAccessorImpl.java:-2


832633

In [4]:
#Import parsing library.
import xml.etree.ElementTree as ET
import numpy as np
from operator import add


In [5]:
def parsePost(line):
    if '  <row'in line:
        try:
            root = ET.fromstring(line)
        except:
            pass
            return ("Empty")
            
        if root != '':
            if "Id" and "FavoriteCount" in root.attrib:
                return(root.attrib["Id"], int(root.attrib["FavoriteCount"]))
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")
            
        

In [6]:
posts = posts_lines.map(parsePost).filter(lambda x: x!= 'Empty')
print posts
print posts.take(50)


PythonRDD[6] at RDD at PythonRDD.scala:43
[('101121', 1), ('101126', 1), ('101144', 1), ('101150', 1), ('101154', 3), ('101178', 2), ('101192', 1), ('101223', 2), ('101233', 1), ('101237', 1), ('101241', 1), ('101243', 0), ('101252', 1), ('101255', 6), ('101274', 28), ('101300', 0), ('101309', 1), ('101313', 0), ('101315', 1), ('101318', 3), ('101349', 1), ('101354', 5), ('101378', 1), ('101383', 2), ('101394', 1), ('101399', 2), ('101405', 1), ('101413', 1), ('101451', 1), ('101457', 2), ('101463', 1), ('101467', 1), ('101496', 1), ('101560', 4), ('101566', 1), ('101575', 0), ('101576', 0), ('101586', 1), ('101590', 2), ('102595', 2), ('102597', 0), ('102610', 1), ('102620', 1), ('102630', 1), ('102638', 1), ('102639', 4), ('102664', 1), ('102669', 1), ('102680', 1), ('102684', 1)]


In [7]:
def parseVote(line): 
    if '  <row'in line:
        try:
            root = ET.fromstring(line)
        except:
            pass
            return("Empty")
        if root != '':
            if "PostId" and "VoteTypeId" in root.attrib:
                vote = root.attrib['VoteTypeId']
                if vote == '2':
                    votes = [1.,0.,1.]
                else:
                    if vote == '3':
                        votes = [0.,1.,1.]
                    else: return("Empty")
                return (root.attrib['PostId'], votes)
            else:
                return("Empty")
        else:
            return("Empty")
    else:
        return("Empty")


In [8]:
votes = votes_lines.map(parseVote).filter(lambda x: x != 'Empty')\
        .reduceByKey(lambda x, y: np.add(x,y))
print votes.take(10)

[('89370', array([ 2.,  0.,  2.])), ('11543', array([ 3.,  0.,  3.])), ('89373', array([ 1.,  1.,  2.])), ('11546', array([ 2.,  0.,  2.])), ('89376', array([ 6.,  1.,  7.])), ('11545', array([ 3.,  0.,  3.])), ('11548', array([ 4.,  0.,  4.])), ('116660', array([ 4.,  0.,  4.])), ('5989', array([ 7.,  0.,  7.])), ('5983', [1.0, 0.0, 1.0])]


In [20]:
joint = posts.join(votes).map(lambda x: (x[1][0], x[1][1]))\
        .reduceByKey(lambda x, y: np.add(x,y)).map(lambda x: (x[0], x[1][0]/x[1][2]))\
        .sortByKey().collect()
print joint[:50]

[(0, 0.94906295050456513), (1, 0.97916290001807993), (2, 0.9868300408435442), (3, 0.98957029620358783), (4, 0.99014389907352651), (5, 0.99210757088570589), (6, 0.99494949494949492), (7, 0.99127239320165361), (8, 0.9952913478516775), (9, 0.99660825325042401), (10, 0.99188191881918819), (11, 0.99182839632277831), (12, 0.99567099567099571), (13, 0.99696048632218848), (14, 0.99332443257676906), (15, 0.99223803363518759), (16, 1.0), (17, 1.0), (18, 0.9985693848354793), (19, 0.99775784753363228), (20, 0.99675324675324672), (21, 0.99378881987577639), (22, 1.0), (23, 0.9945205479452055), (24, 1.0), (25, 1.0), (26, 0.98417721518987344), (27, 0.99619771863117867), (28, 0.99516908212560384), (29, 0.99728260869565222), (30, 0.99543378995433784), (31, 0.99337748344370858), (32, 1.0), (33, 1.0), (34, 1.0), (35, 1.0), (36, 1.0), (37, 0.99099099099099097), (38, 1.0), (39, 0.99180327868852458), (40, 1.0), (41, 1.0), (42, 1.0), (44, 1.0), (45, 1.0), (47, 1.0), (48, 1.0), (49, 1.0), (50, 1.0), (52, 1.0)]

In [ ]:
[(20, 0.9952153110047847)] * 50